In [1]:
!pip -q install datasets apex pyramid tokenizers
!git clone https://github.com/NVIDIA/apex
%cd apex
!python3 setup.py install
import numpy as np
from transformers import BertTokenizer, BertConfig, BertModel
from datasets import load_dataset
import random
import pyramid
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix, csc_matrix, vstack
from tqdm import tqdm
from tokenizers import BertWordPieceTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import torch
from transformers import AdamW
from apex import amp
from torch import nn
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... don

In [2]:
#VARS

levels = []
eps = 1e-4
eval_step = 20000
seed = 42
max_leaf = 20
BATCH = 16
EPOCH = 20
max_len = 512
group_y_candidate_num = 3000
swa_warmup = 10
swa_step = 100
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
labels = []
texts = []
dataType = []
label_map = {}

In [3]:
from google.colab import drive
import os
if (os.path.exists('/content/drive') == 0):
  drive.mount('/content/drive')
# drive_contents = os.listdir('/content/drive/MyDrive/EUR-Lex')
# print(drive_contents)
fext = '_texts.txt'
with open(f'/content/drive/MyDrive/EUR-Lex/train{fext}') as f:
    for i in tqdm(f):
        texts.append(i.replace('\n', ''))
        dataType.append('train')

with open(f'/content/drive/MyDrive/EUR-Lex/test{fext}') as f:
    for i in tqdm(f):
        texts.append(i.replace('\n', ''))
        dataType.append('test')

with open(f'/content/drive/MyDrive/EUR-Lex/train_labels.txt') as f:
    for i in tqdm(f):
        for l in i.replace('\n', '').split():
            label_map[l] = 0
        labels.append(i.replace('\n', ''))


with open(f'/content/drive/MyDrive/EUR-Lex/test_labels.txt') as f:
    print(len(label_map))
    for i in tqdm(f):
        for l in i.replace('\n', '').split():
            label_map[l] = 0
        labels.append(i.replace('\n', ''))
    print(len(label_map))

assert len(texts) == len(labels) == len(dataType)

df_row = {'text': texts, 'label': labels, 'dataType': dataType}

for i, k in enumerate(sorted(label_map.keys())):
    label_map[k] = i
df = pd.DataFrame(df_row)
print(len(label_map))

Mounted at /content/drive


15449it [00:04, 3475.03it/s]
3865it [00:01, 2338.88it/s]
15449it [00:00, 23794.11it/s]


3801


3865it [00:00, 4546.93it/s]

3956
3956


In [4]:
class MDataset(Dataset):
    def __init__(self, df, mode, tokenizer, label_map, max_length,
                 token_type_ids=None, group_y=None, candidates_num=None):
        assert mode in ["train", "valid", "test"]
        self.mode = mode
        self.df, self.n_labels, self.label_map = df[df.dataType == self.mode], len(label_map), label_map
        self.len = len(self.df)
        self.tokenizer, self.max_length, self.group_y = tokenizer, max_length, group_y
        self.multi_group = False
        self.token_type_ids = token_type_ids
        self.candidates_num = candidates_num

        if group_y is not None:
            # print("GROUP Y")
            # print(group_y)
            # group y mode
            self.candidates_num, self.group_y, self.n_group_y_labels = candidates_num, [], group_y.shape[0]
            # print(len(label_map))
            self.map_group_y = np.empty(len(label_map), dtype=np.compat.long)
            # ctr = 0
            for idx, labels in enumerate(group_y):
                self.group_y.append([])
                for label in labels:
                    # ctr += 1
                    self.group_y[-1].append(label_map[label])
                self.map_group_y[self.group_y[-1]] = idx
                self.group_y[-1]  = np.array(self.group_y[-1],dtype=int)
            # print(self.map_group_y)
            # assert ctr == 3956
            self.group_y = np.array(self.group_y,dtype=object)
        else:
          self.group_y = None

    def __getitem__(self, idx):
        # print(idx)
        # print("here")
        # print(self.group_y == None)
        max_len = self.max_length
        review = self.df.text.values[idx].lower()
        labels = [self.label_map[i] for i in self.df.label.values[idx].split() if i in self.label_map]
        # print("len of labels is " + str(len(labels)))
        review = ' '.join(review.split()[:max_len])

        text = review
        if self.token_type_ids is not None:
            input_ids = self.token_type_ids[idx]
            if input_ids[-1] == 0:
                input_ids = input_ids[input_ids != 0]
            input_ids = input_ids.tolist()
        elif hasattr(self.tokenizer, 'encode_plus'):
            input_ids = self.tokenizer.encode(
                'filling empty' if len(text) == 0 else text,
                add_special_tokens=True,
                max_length=max_len
            )
        else:
            # fast
            input_ids = self.tokenizer.encode(
                'filling empty' if len(text) == 0 else text,
                add_special_tokens=True
            ).ids

        if len(input_ids) == 0:
            print('zero string')
            assert 0
        if len(input_ids) > self.max_length:
            input_ids[self.max_length-1] = input_ids[-1]
            input_ids = input_ids[:self.max_length]

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        padding_length = self.max_length - len(input_ids)
        input_ids = input_ids + ([0] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)

        input_ids = torch.tensor(input_ids)
        attention_mask = torch.tensor(attention_mask)
        token_type_ids = torch.tensor(token_type_ids)


        if self.group_y is not None:
            # print("DAAAAMN")
            label_ids = torch.zeros(self.n_labels)
            label_ids = label_ids.scatter(0, torch.tensor(labels),
                                          torch.tensor([1.0 for i in labels]))
            group_labels = self.map_group_y[labels].tolist()
            group_labels = [label for label in group_labels if label < self.n_group_y_labels]
            # print(self.map_group_y)
            if self.multi_group:
                group_labels = np.concatenate(group_labels).tolist()
            # print(group_labels)
            group_label_ids = torch.zeros(self.n_group_y_labels)
            group_label_ids = group_label_ids.scatter(0, torch.tensor(group_labels),
                                      torch.tensor([1.0 for i in group_labels]))
            # except Exception as e:
            #   print(group_labels)
            candidates = np.concatenate(self.group_y[group_labels], axis=0)

            if len(candidates) < self.candidates_num:
                sample = np.random.randint(self.n_group_y_labels, size=self.candidates_num - len(candidates))
                candidates = np.concatenate([candidates, sample])
            elif len(candidates) > self.candidates_num:
                candidates = np.random.choice(candidates, self.candidates_num, replace=False)

            if self.mode == 'train':
                return input_ids, attention_mask, token_type_ids,\
                    label_ids[candidates], group_label_ids, candidates
            else:
                return input_ids, attention_mask, token_type_ids,\
                    label_ids, group_label_ids, candidates
            # root fc layer
            group_labels = layers_group_labels[0]
            group_label_ids = torch.zeros(len(self.map_children[0]))
            group_label_ids = group_label_ids.scatter(0, torch.tensor(group_labels),
                                                      torch.tensor([1.0 for i in group_labels]))
            layers_group_labels_ids.append(group_label_ids)

            if self.mode == 'train':
                return input_ids, attention_mask, token_type_ids, \
                    layers_group_labels_ids[::-1], layers_candidates[::-1]
            else:
                return input_ids, attention_mask, token_type_ids, layers_group_labels + [labels]

        label_ids = torch.zeros(self.n_labels)
        label_ids = label_ids.scatter(0, torch.tensor(labels),
                                      torch.tensor([1.0 for i in labels]))
        return input_ids, attention_mask, token_type_ids, label_ids

    def __len__(self):
        return self.len

In [5]:
model_config = BertConfig.from_pretrained('bert-base-uncased')
model_config.output_hidden_states = True
model_bert = BertModel.from_pretrained('bert-base-uncased', config=model_config)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
from sklearn.datasets import load_svmlight_file
def build_tree_by_level(eps: float, max_leaf: int, levels: list):
  print('Clustering')
  fext = '.txt'
  sparse_x_train, _ = load_svmlight_file(f'/content/drive/MyDrive/EUR-Lex/train{fext}', multilabel=True)
  sparse_labels_train = [i.replace('\n', '').split() for i in open('/content/drive/MyDrive/EUR-Lex/train_labels.txt')]
  sparse_x_test, _ = load_svmlight_file(f'/content/drive/MyDrive/EUR-Lex/test{fext}', multilabel=True)
  sparse_labels_test = [i.replace('\n', '').split() for i in open('/content/drive/MyDrive/EUR-Lex/test_labels.txt')]
  sparse_labels = sparse_labels_train + sparse_labels_test
  sparse_x = vstack([sparse_x_train, sparse_x_test])
  sparse_x, sparse_y = normalize(sparse_x), np.array(sparse_labels,dtype=object)
  mlb = MultiLabelBinarizer(sparse_output=True)
  sparse_y = mlb.fit_transform(sparse_y)
  # sparse_x, sparse_labels = get_sparse_feature(sparse_data_x, sparse_data_y)
  # mlb = MultiLabelBinarizer(sparse_output=True)
  # sparse_y = mlb.fit_transform(sparse_labels)
  # joblib.dump(mlb, groups_path+'mlb')
  print('Getting Labels Feature')
  labels_f = normalize(csr_matrix(sparse_y.T) @ csc_matrix(sparse_x))
  print(F'Start Clustering {levels}')
  levels, q = [2**x for x in levels], None
      # if os.path.exists(F'{groups_path}-Level-{i}.npy'):
      #     print(F'{groups_path}-Level-{i}.npy')
      #     labels_list = np.load(F'{groups_path}-Level-{i}.npy', allow_pickle=True)
      #     q = [(labels_i, labels_f[labels_i]) for labels_i in labels_list]
      #     break
  if q is None:
    q = [(np.arange(labels_f.shape[0]), labels_f)]
  # print(q)
  while q:
    labels_list = np.asarray([x[0] for x in q],dtype=object)
    assert sum(len(labels) for labels in labels_list) == labels_f.shape[0]
    # print(len(labels_list))
    # print(labels_list)
    # break
    if len(labels_list) in levels:
        level = levels.index(len(labels_list))
        print(F'Finish Clustering Level-{level}')
        # np.save(F'{groups_path}-Level-{level}.npy', np.asarray(labels_list))
    else:
        print(F'Finish Clustering {len(labels_list)}')
    next_q = []
    for node_i, node_f in q:
        # print(len(node_i))
        # print(len(node_f))
        # print(node_i)
        # break
        if len(node_i) > max_leaf:
            next_q += list(split_node(node_i, node_f, eps))
        else:
          np.save('last.npy', np.asarray(labels_list))
    q = next_q
  print('Finish Clustering')
  # print(labels_list)
  # print(len(labels_list))
  # for i in labels_list:
  #   # print(len(i))
  return mlb, np.asarray(labels_list,dtype=object)

def split_node(labels_i: np.ndarray, labels_f: csr_matrix, eps: float):
  n = len(labels_i)
  c1, c2 = np.random.choice(np.arange(n), 2, replace=False)
  centers, old_dis, new_dis = labels_f[[c1, c2]].toarray(), -10000.0, -1.0
  l_labels_i, r_labels_i = None, None
  while new_dis - old_dis >= eps:
      dis = labels_f @ centers.T  # N, 2
      partition = np.argsort(dis[:, 1] - dis[:, 0])
      l_labels_i, r_labels_i = partition[:n//2], partition[n//2:]
      old_dis, new_dis = new_dis, (dis[l_labels_i, 0].sum() + dis[r_labels_i, 1].sum()) / n
      centers = normalize(np.asarray([np.squeeze(np.asarray(labels_f[l_labels_i].sum(axis=0))),
                                      np.squeeze(np.asarray(labels_f[r_labels_i].sum(axis=0)))]))
  return (labels_i[l_labels_i], labels_f[l_labels_i]), (labels_i[r_labels_i], labels_f[r_labels_i])

In [7]:
mlb, groups = build_tree_by_level(eps, max_leaf, levels)
groups = np.load('last.npy', allow_pickle=True)
new_group = []
for group in groups:
  new_group.append([mlb.classes_[i] for i in group])
new_group = np.array(new_group,dtype=object)

Clustering
Getting Labels Feature
Start Clustering []
Finish Clustering 1
Finish Clustering 2
Finish Clustering 4
Finish Clustering 8
Finish Clustering 16
Finish Clustering 32
Finish Clustering 64
Finish Clustering 128
Finish Clustering 256
Finish Clustering


In [8]:
# candidates_num, group_y, n_group_y_labels = 3000, [], new_group.shape[0]
# map_group_y = np.empty(len(label_map), dtype=np.compat.long)
# for idx, labels in enumerate(group_y):
#     group_y.append([])
#     for label in labels:
#         group_y[-1].append(label_map[label])
#     map_group_y[group_y[-1]] = idx
#     group_y[-1]  = np.array(group_y[-1])
# group_y = np.array(group_y)
# print(map_group_y)
# print(group_y)

In [9]:
class LightXML(nn.Module):
    def __init__(self,model , n_labels, group_y=None, bert='bert-base', feature_layers=5, dropout=0.5, update_count=1,
                 candidates_topk=10,
                 use_swa=True, swa_warmup_epoch=10, swa_update_step=200, hidden_dim=300):
        super(LightXML, self).__init__()

        self.use_swa = use_swa
        self.swa_warmup_epoch = swa_warmup_epoch
        self.swa_update_step = swa_update_step
        self.swa_state = {}

        self.update_count = update_count

        self.candidates_topk = candidates_topk

        print('swa', self.use_swa, self.swa_warmup_epoch, self.swa_update_step, self.swa_state)
        print('update_count', self.update_count)

        self.bert_name, self.bert = bert, model
        self.feature_layers, self.drop_out = feature_layers, nn.Dropout(dropout)

        self.group_y = group_y
        if self.group_y is not None:
            self.group_y_labels = group_y.shape[0]
            print('hidden dim:',  hidden_dim)
            print('label goup numbers:',  self.group_y_labels)

            self.l0 = nn.Linear(self.feature_layers*self.bert.config.hidden_size, self.group_y_labels)
            # hidden bottle layer
            self.l1 = nn.Linear(self.feature_layers*self.bert.config.hidden_size, hidden_dim)
            self.embed = nn.Embedding(n_labels, hidden_dim)
            nn.init.xavier_uniform_(self.embed.weight)
        else:
            self.l0 = nn.Linear(self.feature_layers*self.bert.config.hidden_size, n_labels)

    def get_candidates(self, group_logits, group_gd=None):
        logits = torch.sigmoid(group_logits.detach())
        if group_gd is not None:
            logits += group_gd
        scores, indices = torch.topk(logits, k=self.candidates_topk)
        scores, indices = scores.cpu().detach().numpy(), indices.cpu().detach().numpy()
        candidates, candidates_scores = [], []
        for index, score in zip(indices, scores):
            candidates.append(self.group_y[index])
            candidates_scores.append([np.full(c.shape, s) for c, s in zip(candidates[-1], score)])
            candidates[-1] = np.concatenate(candidates[-1])
            candidates_scores[-1] = np.concatenate(candidates_scores[-1])
        max_candidates = max([i.shape[0] for i in candidates])
        candidates = np.stack([np.pad(i, (0, max_candidates - i.shape[0]), mode='edge') for i in candidates])
        candidates_scores = np.stack([np.pad(i, (0, max_candidates - i.shape[0]), mode='edge') for i in candidates_scores])
        return indices, candidates, candidates_scores

    def forward(self, input_ids, attention_mask, token_type_ids,
                labels=None, group_labels=None, candidates=None):
        is_training = labels is not None

        outs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )[-1]

        out = torch.cat([outs[-i][:, 0] for i in range(1, self.feature_layers+1)], dim=-1)
        out = self.drop_out(out)
        group_logits = self.l0(out)
        if self.group_y is None:
            logits = group_logits
            if is_training:
                loss_fn = torch.nn.BCEWithLogitsLoss()
                loss = loss_fn(logits, labels)
                return logits, loss
            else:
                return logits

        if is_training:
            l = labels.to(dtype=torch.bool)
            target_candidates = torch.masked_select(candidates, l).detach().cpu()
            target_candidates_num = l.sum(dim=1).detach().cpu()
        groups, candidates, group_candidates_scores = self.get_candidates(group_logits,
                                                                          group_gd=group_labels if is_training else None)
        if is_training:
            bs = 0
            new_labels = []
            for i, n in enumerate(target_candidates_num.numpy()):
                be = bs + n
                c = set(target_candidates[bs: be].numpy())
                c2 = candidates[i]
                new_labels.append(torch.tensor([1.0 if i in c else 0.0 for i in c2 ]))
                if len(c) != new_labels[-1].sum():
                    s_c2 = set(c2)
                    for cc in list(c):
                        if cc in s_c2:
                            continue
                        for j in range(new_labels[-1].shape[0]):
                            if new_labels[-1][j].item() != 1:
                                c2[j] = cc
                                new_labels[-1][j] = 1.0
                                break
                bs = be
            labels = torch.stack(new_labels).cuda()
        candidates, group_candidates_scores =  torch.LongTensor(candidates).cuda(), torch.Tensor(group_candidates_scores).cuda()

        emb = self.l1(out)
        embed_weights = self.embed(candidates) # N, sampled_size, H
        emb = emb.unsqueeze(-1)
        logits = torch.bmm(embed_weights, emb).squeeze(-1)

        if is_training:
            loss_fn = torch.nn.BCEWithLogitsLoss()
            loss = loss_fn(logits, labels) + loss_fn(group_logits, group_labels)
            return logits, loss
        else:
            candidates_scores = torch.sigmoid(logits)
            candidates_scores = candidates_scores * group_candidates_scores
            return group_logits, candidates, candidates_scores

    def swa_init(self):
        self.swa_state = {'models_num': 1}
        for n, p in self.named_parameters():
            self.swa_state[n] = p.data.cpu().clone().detach()

    def swa_step(self):
        if 'models_num' not in self.swa_state:
            return
        self.swa_state['models_num'] += 1
        beta = 1.0 / self.swa_state['models_num']
        with torch.no_grad():
            for n, p in self.named_parameters():
                self.swa_state[n].mul_(1.0 - beta).add_(beta, p.data.cpu())

    def swa_swap_params(self):
        if 'models_num' not in self.swa_state:
            return
        for n, p in self.named_parameters():
            self.swa_state[n], p.data =  self.swa_state[n].cpu(), p.data.cpu()
            self.swa_state[n], p.data =  p.data.cpu(), self.swa_state[n].cuda()

    def get_tokenizer(self):
        print('load bert-base-uncased tokenizer')
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        return tokenizer

    def get_accuracy(self, candidates, logits, labels):
        if candidates is not None:
            candidates = candidates.detach().cpu()
        scores, indices = torch.topk(logits.detach().cpu(), k=10)

        acc1, acc3, acc5, total = 0, 0, 0, 0
        for i, l in enumerate(labels):
            l = set(np.nonzero(l)[0])

            if candidates is not None:
                labels = candidates[i][indices[i]].numpy()
            else:
                labels = indices[i, :5].numpy()

            acc1 += len(set([labels[0]]) & l)
            acc3 += len(set(labels[:3]) & l)
            acc5 += len(set(labels[:5]) & l)
            total += 1

        return total, acc1, acc3, acc5

    def one_epoch(self, epoch, dataloader, optimizer,
                  mode='train', eval_loader=None, eval_step=20000):

        bar = tqdm(total=len(dataloader))
        p1, p3, p5 = 0, 0, 0
        g_p1, g_p3, g_p5 = 0, 0, 0
        total, acc1, acc3, acc5 = 0, 0, 0, 0
        g_acc1, g_acc3, g_acc5 = 0, 0, 0
        train_loss = 0

        if mode == 'train':
            self.train()
        else:
            self.eval()

        if self.use_swa and epoch == self.swa_warmup_epoch and mode == 'train':
            self.swa_init()

        if self.use_swa and mode == 'eval':
            self.swa_swap_params()

        pred_scores, pred_labels = [], []
        bar.set_description(f'{mode}-{epoch}')

        with torch.set_grad_enabled(mode == 'train'):
            # print(dataloader)
            for step, data in enumerate(dataloader):
                # print(step)
                # print(data)
                # print("FFFFF")
                batch = tuple(t for t in data)
                # print(len(batch))
                have_group = len(batch) > 4
                inputs = {'input_ids':      batch[0].cuda(),
                          'attention_mask': batch[1].cuda(),
                          'token_type_ids': batch[2].cuda()}
                if mode == 'train':
                    inputs['labels'] = batch[3].cuda()
                    if self.group_y is not None:
                        # print(self.group_y)
                        # print("HEREEEEEEEEEEEEEE\n\n\n\n\n\n\n\n\n\n\n\n")
                        inputs['group_labels'] = batch[4].cuda()
                        inputs['candidates'] = batch[5].cuda()

                outputs = self(**inputs)

                bar.update(1)

                if mode == 'train':
                    loss = outputs[1]
                    loss /= self.update_count
                    train_loss += loss.item()

                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()

                    if step % self.update_count == 0:
                        optimizer.step()
                        self.zero_grad()

                    if step % eval_step == 0 and eval_loader is not None and step != 0:
                        results = self.one_epoch(epoch, eval_loader, optimizer, mode='eval')
                        p1, p3, p5 = results[3:6]
                        g_p1, g_p3, g_p5 = results[:3]
                        # if self.group_y is not None:
                        #     log.log(f'{epoch:>2} {step:>6}: {p1:.4f}, {p3:.4f}, {p5:.4f}'
                        #             f' {g_p1:.4f}, {g_p3:.4f}, {g_p5:.4f}')
                        # else:
                        #     log.log(f'{epoch:>2} {step:>6}: {p1:.4f}, {p3:.4f}, {p5:.4f}')
                        # NOTE: we don't reset model to train mode and keep model in eval mode
                        # which means all dropout will be remove after `eval_step` in every epoch
                        # this tricks makes LightXML converge fast
                        # self.train()

                    if self.use_swa and step % self.swa_update_step == 0:
                        self.swa_step()

                    bar.set_postfix(loss=loss.item())
                elif self.group_y is None:
                    logits = outputs
                    if mode == 'eval':
                        labels = batch[3]
                        _total, _acc1, _acc3, _acc5 =  self.get_accuracy(None, logits, labels.cpu().numpy())
                        total += _total; acc1 += _acc1; acc3 += _acc3; acc5 += _acc5
                        p1 = acc1 / total
                        p3 = acc3 / total / 3
                        p5 = acc5 / total / 5
                        bar.set_postfix(p1=p1, p3=p3, p5=p5)
                    elif mode == 'test':
                        pred_scores.append(logits.detach().cpu())
                else:
                    group_logits, candidates, logits = outputs

                    if mode == 'eval':
                        # print(batch)
                        labels = batch[3]
                        group_labels = batch[4]

                        _total, _acc1, _acc3, _acc5 = self.get_accuracy(candidates, logits, labels.cpu().numpy())
                        total += _total; acc1 += _acc1; acc3 += _acc3; acc5 += _acc5
                        p1 = acc1 / total
                        p3 = acc3 / total / 3
                        p5 = acc5 / total / 5

                        _, _g_acc1, _g_acc3, _g_acc5 = self.get_accuracy(None, group_logits, group_labels.cpu().numpy())
                        g_acc1 += _g_acc1; g_acc3 += _g_acc3; g_acc5 += _g_acc5
                        g_p1 = g_acc1 / total
                        g_p3 = g_acc3 / total / 3
                        g_p5 = g_acc5 / total / 5
                        bar.set_postfix(p1=p1, p3=p3, p5=p5, g_p1=g_p1, g_p3=g_p3, g_p5=g_p5)
                    elif mode == 'test':
                        _scores, _indices = torch.topk(logits.detach().cpu(), k=100)
                        _labels = torch.stack([candidates[i][_indices[i]] for i in range(_indices.shape[0])], dim=0)
                        pred_scores.append(_scores.cpu())
                        pred_labels.append(_labels.cpu())


        if self.use_swa and mode == 'eval':
            self.swa_swap_params()
        bar.close()

        if mode == 'eval':
            return g_p1, g_p3, g_p5, p1, p3, p5
        elif mode == 'test':
            return torch.cat(pred_scores, dim=0).numpy(), torch.cat(pred_labels, dim=0).numpy() if len(pred_labels) != 0 else None
        elif mode == 'train':
            return train_loss

def train(model, df, label_map,lr,epoch,group_y,max_len,group_y_candidate_num,batch,epochs,valid,eval_step,tokenizer):
    tokenizer = model.get_tokenizer()

    # group_y = load_group(args.dataset, args.group_y_group)
    train_d = MDataset(df, 'train', tokenizer, label_map, max_len, group_y=group_y,
                           candidates_num=group_y_candidate_num)#, token_type_ids=token_type_ids)
    test_d = MDataset(df, 'test', tokenizer, label_map, max_len, group_y=group_y,
                           candidates_num=group_y_candidate_num)#, token_type_ids=token_type_ids)

    train_d.tokenizer = BertWordPieceTokenizer(
                "/content/drive/MyDrive/EUR-Lex/bert-base-uncased-vocab.txt",
                lowercase=True)
    test_d.tokenizer = BertWordPieceTokenizer(
                "/content/drive/MyDrive/EUR-Lex/bert-base-uncased-vocab.txt",
                lowercase=True)
    # print(train_d)
    # print("end")
    trainloader = DataLoader(train_d, batch_size=batch, num_workers=2,
                              shuffle=True)
    testloader = DataLoader(test_d, batch_size=batch, num_workers=2,
                            shuffle=False)
    valid_d = MDataset(df, 'valid', tokenizer, label_map, max_len)
    validloader = DataLoader(valid_d, batch_size=batch, num_workers=2,
                              shuffle=False)

    model.cuda()
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)#, eps=1e-8)

    model, optimizer = amp.initialize(model, optimizer, opt_level="O1")
    max_only_p5 = 0
    for epoch in range(0, epoch+5):
        train_loss = model.one_epoch(epoch, trainloader, optimizer, mode='train',
                                     eval_loader=testloader,
                                     eval_step=eval_step)
        ev_result = model.one_epoch(epoch, testloader, optimizer, mode='eval')

        g_p1, g_p3, g_p5, p1, p3, p5 = ev_result

        # log_str = f'{epoch:>2}: {p1:.4f}, {p3:.4f}, {p5:.4f}, train_loss:{train_loss}'
        # if args.dataset in ['wiki500k', 'amazon670k']:
        #     log_str += f' {g_p1:.4f}, {g_p3:.4f}, {g_p5:.4f}'
        # if args.valid:
        #     log_str += ' valid'
        # LOG.log(log_str)

        if max_only_p5 < p5:
            max_only_p5 = p5
            # model.save_model(f'models/model-{get_exp_name()}.bin')

        if epoch >= epochs + 5 and max_only_p5 != p5:
            break

In [10]:
train_df, valid_df = train_test_split(df[df['dataType'] == 'train'],
                                              test_size=4000,
                                              random_state=1240)
df.iloc[valid_df.index.values, 2] = 'valid'
print('valid size', len(df[df['dataType'] == 'valid']))
print(f'load EURLEX dataset with '
          f'{len(df[df.dataType =="train"])} train {len(df[df.dataType =="test"])} test with {len(label_map)} labels done')

valid size 4000
load EURLEX dataset with 11449 train 3865 test with 3956 labels done


In [11]:
_group_y = []
for idx, labels in enumerate(new_group):
  _group_y.append([])
  for label in labels:
    _group_y[-1].append(label_map[label])
    # _group_y[-1] = np.array(_group_y[-1])
for i in range(len(_group_y)):
  _group_y[i] = np.array(_group_y[i])
group_y_init = np.array(_group_y,dtype=object)
# group_y_init

In [ ]:
# print(group_y)
model = LightXML(model_bert,n_labels=len(label_map), group_y=group_y_init,
                          update_count=1,
                          use_swa=True, swa_warmup_epoch=4, swa_update_step=3000,
                          candidates_topk=32,
                          hidden_dim=500)
# model = LightXML(model_bert,n_labels=len(label_map),
#                          update_count=1,
#                          use_swa=True, swa_warmup_epoch=10, swa_update_step=200)
train(model, df, label_map,eps,EPOCH,new_group,max_len,group_y_candidate_num,BATCH,EPOCH,False,eval_step,tokenizer)

swa True 4 3000 {}
update_count 1
hidden dim: 500
label goup numbers: 256
load bert-base-uncased tokenizer


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/content/apex/apex/__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
  warnings.warn(msg, DeprecatedFeatureWarning)
/content/apex/apex/amp/scaler.py:56: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  self._overflow_buf = torch.cuda.IntTensor([0])


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


train-4:   0%|          | 1/716 [00:00<11:20,  1.05it/s]/content/apex/apex/amp/wrap.py:111: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  return orig_fn(arg0, *new_args, **kwargs)
train-6:  57%|█████▋    | 409/716 [04:13<03:08,  1.63it/s, loss=0.0488]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


train-17:  77%|███████▋  | 548/716 [05:39<01:43,  1.62it/s, loss=0.0154]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1048576.0


train-20:  94%|█████████▍| 676/716 [06:58<00:24,  1.63it/s, loss=0.0178] 

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1048576.0


train-22:  64%|██████▍   | 458/716 [04:43<02:38,  1.63it/s, loss=0.00887]

In [ ]:
train = [i.replace('\n', '').split() for i in open('/content/drive/MyDrive/EUR-Lex/train_labels.txt')]
test = [i.replace('\n', '').split() for i in open('/content/drive/MyDrive/EUR-Lex/test_labels.txt')]

def get_unique_strings(list_of_lists):
    # Flatten the list of lists into a single list
    flat_list = [string for sublist in list_of_lists for string in sublist]

    # Use set to get unique strings
    unique_strings = list(set(flat_list))

    return unique_strings

train = get_unique_strings(train)
test = get_unique_strings(test)


In [ ]:
def find_unique_strings(list1, list2):
    # Convert both lists to sets for efficient membership checking
    set1 = set(list1)
    set2 = set(list2)

    # Find the difference (strings in set2 but not in set1)
    unique_strings = list(set2 - set1)

    return unique_strings
len(find_unique_strings(train,test))